<a href="https://colab.research.google.com/github/mateusduarte-max/Projetos_Python/blob/main/26_Municipios_Brasil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install beautifulsoup4
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time
# Montar Drive
from google.colab import drive
drive.mount('/content/drive')


import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
from gspread_dataframe import set_with_dataframe


# Classe dados 
!cp /content/drive/MyDrive/Colab_Notebooks/pacotes_modulos/modulos_colab.py /content
from modulos_colab import dados
!cp /content/drive/MyDrive/Colab_Notebooks/Autenticacoes/chaves_tokens.py /content
%run chaves_tokens.py

In [ ]:
# Base municipios e estados
base_municipios = dados.busca_tabela_drive('municipios_brasil', 'base')
base_municipios['municipio'] = base_municipios['municipio'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.replace(' ','_') 
base_municipios['estado'] = base_municipios['estado'].str.lower()
municipio = base_municipios['municipio'].to_list()
estado = base_municipios['estado'].to_list()


In [ ]:

for m, e in zip(municipio, estado):
    html = requests.get(f"https://www.feriados.com.br/feriados-{m}-{e}.php?ano=2022").content
    soup = BeautifulSoup(html, 'html.parser')
    feriados = [t.text.strip() for t in soup.find_all("span", attrs={'class': 'style_lista_feriados'})]
    df = pd.DataFrame(feriados, columns=['col'])
    divisao = df["col"].str.split("-", n = 1, expand = True)
    df["Data"] = divisao[0]
    df["Desricao"] = divisao[1]
    df['municipio'] = m
    df['estado'] = e
    def linhas():
        arquivo = gc.open('base_final_municipios')
        # abrir planilha 'aba'
        aba = arquivo.worksheet('Página1')
        # colocar a aba dentro de um data_frame
        df1 = pd.DataFrame(aba.get_all_records())
        #Visualização
        numero = df1['col'].count()+2
        n = int(numero)
        return n
     
    def input_drive():
        #Abrir planilha e primeira aba
        time.sleep(3)
        pagina = gc.open('base_final_municipios').sheet1
        # Limpar base
        #pagina.clear()
        # Importar dataframe na planilha
        return set_with_dataframe(pagina, df, row=linhas(), include_column_header=False)
    input_drive()
    time.sleep(3)
    